<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB30_Approche_ReAct_pour_agents_de_raisonnement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: approche ReAct pour Reasoning Agents

**Objectif**: Implémenter le pattern ReAct où le LLM alterne entre pensée (Thought) et action (Action).

**Durée estimée**: 20-30 minutes

**Livrable**: Un agent ReAct fonctionnel avec plusieurs outils (calculatrice, base de connaissances, extensions).

---

## 📋 Prérequis

- Compte OpenAI avec clé API
- Compréhension de base des LLMs
- Connaissance Python de base

---

## Step 1: Setup :

Installation des dépendances nécessaires.

In [1]:
# Installation des packages
!pip install -q openai python-dotenv

In [4]:
import os

# Option 1: Utiliser les Secrets de Colab (recommandé)
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ Clé API chargée depuis les Secrets Colab")
except:
    # Option 2: Définir manuellement (décommentez et ajoutez votre clé)
    # os.environ['OPENAI_API_KEY'] = "sk-votre-cle-api-ici"
    print("⚠️ Veuillez configurer votre clé API OpenAI")
    print("   Utilisez les Secrets Colab ou décommentez la ligne ci-dessus")


✅ Clé API chargée depuis les Secrets Colab


## Step 2: Definition des Tools

Définition des outils de base: calculatrice et base de connaissances.

In [5]:
import math

def calculator(expression: str):
    """
    Évalue une expression mathématique.

    Args:
        expression: Expression mathématique sous forme de chaîne

    Returns:
        Résultat du calcul ou message d'erreur
    """
    try:
        # Ajouter le module math au contexte d'évaluation pour les fonctions avancées
        result = eval(expression, {"__builtins__": {}}, {"math": math})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

# Base de connaissances simple
knowledge_base = {
    "LangChain": "A framework for building LLM-powered agents.",
    "RAG": "Retrieval-Augmented Generation, which uses external knowledge to improve answers.",
    "ReAct": "A reasoning pattern that alternates between Reasoning and Acting to solve problems.",
    "OpenAI": "An AI research company that created GPT models and ChatGPT.",
    "Agent": "An autonomous system that can perceive its environment, make decisions, and take actions."
}

def lookup(term: str):
    """
    Recherche un terme dans la base de connaissances.

    Args:
        term: Terme à rechercher

    Returns:
        Définition ou message d'absence
    """
    return knowledge_base.get(term, f"No info found for {term}")

# Tests unitaires des outils
print("🧪 Tests des outils:")
print(f"calculator('15*3+120/6'): {calculator('15*3+120/6')}")
print(f"lookup('LangChain'): {lookup('LangChain')}")
print(f"lookup('Unknown'): {lookup('Unknown')}")

🧪 Tests des outils:
calculator('15*3+120/6'): 65.0
lookup('LangChain'): A framework for building LLM-powered agents.
lookup('Unknown'): No info found for Unknown


## Step 3: ReAct Loop Structure

Implémentation de la boucle ReAct: **Reasoning → Action → Observation → Final Answer**

In [6]:
from openai import OpenAI
import os
import re

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def react_agent(query, max_turns=5, verbose=True):
    """
    Agent ReAct qui alterne entre raisonnement et action.

    Args:
        query: Question posée à l'agent
        max_turns: Nombre maximum d'itérations
        verbose: Afficher la trace de raisonnement

    Returns:
        Réponse finale de l'agent
    """
    system_prompt = """You are a ReAct agent. You must think step by step and take actions when needed.

Format your response as follows:
Thought: [Your reasoning about what to do next]
Action: [tool_name[input]] OR skip if no action needed
Observation: [Will be provided by the system after action]
Final Answer: [Only when you have enough information to answer]

Available tools:
- calculator[expression]: Evaluate mathematical expressions (e.g., calculator[2+2])
- lookup[term]: Search for a term in the knowledge base

Rules:
1. Always start with a Thought
2. If you need information, use an Action
3. Wait for Observation before continuing
4. Only provide Final Answer when you're certain
"""

    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]

    if verbose:
        print("="*80)
        print(f"🤔 Question: {query}")
        print("="*80)

    final_answer = None

    for turn in range(max_turns):
        if verbose:
            print(f"\n--- Tour {turn + 1}/{max_turns} ---")

        # Appel au LLM
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation,
            temperature=0
        )
        reply = resp.choices[0].message.content

        if verbose:
            print(f"\n{reply}")

        # Vérifier si l'agent a donné une réponse finale
        if "Final Answer:" in reply:
            final_answer = reply.split("Final Answer:")[1].strip()
            if verbose:
                print("\n" + "="*80)
                print(f"✅ Réponse finale: {final_answer}")
                print("="*80)
            break

        # Extraire et exécuter l'action
        if "Action:" in reply:
            action_line = [line for line in reply.split("\n") if "Action:" in line][0]

            # Parser l'action
            if "calculator[" in action_line:
                match = re.search(r'calculator\[(.+?)\]', action_line)
                if match:
                    expr = match.group(1)
                    obs = calculator(expr)
                    if verbose:
                        print(f"\n🔧 Exécution: calculator[{expr}]")
                        print(f"📊 Observation: {obs}")
                else:
                    obs = "Error: Could not parse calculator expression"

            elif "lookup[" in action_line:
                match = re.search(r'lookup\[(.+?)\]', action_line)
                if match:
                    term = match.group(1).strip()
                    obs = lookup(term)
                    if verbose:
                        print(f"\n🔍 Recherche: lookup[{term}]")
                        print(f"📊 Observation: {obs}")
                else:
                    obs = "Error: Could not parse lookup term"

            else:
                obs = "Unknown tool. Available tools: calculator, lookup"

            # Ajouter l'échange à la conversation
            conversation.append({"role": "assistant", "content": reply})
            conversation.append({"role": "user", "content": f"Observation: {obs}"})
        else:
            # Pas d'action détectée mais pas de Final Answer non plus
            conversation.append({"role": "assistant", "content": reply})
            if verbose:
                print("\n⚠️ Aucune action détectée. Continuation...")

    if final_answer is None:
        final_answer = "Max turns reached without final answer."
        if verbose:
            print(f"\n⚠️ {final_answer}")

    return final_answer

print("✅ Agent ReAct défini et prêt à l'emploi")

✅ Agent ReAct défini et prêt à l'emploi


## Step 4: Test avec une question de mathématiques :

Test de l'agent avec une question mathématique.

In [8]:
# Test 1: Calcul mathématique simple
result = react_agent("What is : 15*3 - 2 + 120/6 + 27 ?")
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: What is : 15*3 - 2 + 120/6 + 27 ?

--- Tour 1/5 ---

Thought: I need to evaluate the mathematical expression step by step. I can use the calculator tool to compute the result of the expression 15*3 - 2 + 120/6 + 27. 
Action: calculator[15*3 - 2 + 120/6 + 27] 
Observation: [Will be provided by the system after action]

🔧 Exécution: calculator[15*3 - 2 + 120/6 + 27]
📊 Observation: 90.0

--- Tour 2/5 ---

Final Answer: 90.0

✅ Réponse finale: 90.0

🎯 Résultat extrait: 90.0


In [9]:
# Test 2: Calcul plus complexe
result = react_agent("Calcule la racine carrée de 144 + 25 multiplié par 4.")
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: Calcule la racine carrée de 144 + 25 multiplié par 4.

--- Tour 1/5 ---

Thought: I need to calculate the expression step by step. First, I will calculate 144 + 25, then multiply the result by 4, and finally take the square root of that result. I will use the calculator tool for this calculation. 

Action: calculator[√(144 + 25) * 4] 

Observation: [Will be provided by the system after action]

🔧 Exécution: calculator[√(144 + 25) * 4]
📊 Observation: Error: invalid character '√' (U+221A) (<string>, line 1)

--- Tour 2/5 ---

Thought: It seems that the square root symbol is not recognized in the calculator tool. I will break down the calculation into parts. First, I will calculate 144 + 25, then multiply that result by 4, and finally take the square root of the final result. 

Action: calculator[144 + 25] 

Observation: [Will be provided by the system after action]

🔧 Exécution: calculator[144 + 25]
📊 Observation: 169

--- Tour 3/5 ---

Thought: Now that I have the result of 

## Step 5: Test avec une base de connaissance :

Test de l'agent avec recherche dans la base de connaissances.

In [10]:
# Test 3: Recherche simple dans la KB
result = react_agent("Explique LangChain en 1 phrase en verlan.")
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: Explique LangChain en 1 phrase en verlan.

--- Tour 1/5 ---

Thought: Je vais chercher des informations sur LangChain pour pouvoir expliquer ce concept en verlan. 
Action: lookup[LangChain] 
Observation: [Will be provided by the system after action]

🔍 Recherche: lookup[LangChain]
📊 Observation: A framework for building LLM-powered agents.

--- Tour 2/5 ---

Thought: Maintenant que j'ai l'information sur LangChain, je vais formuler une phrase en verlan pour expliquer ce qu'est LangChain. 
Action: skip 
Observation: [Will be provided by the system after action] 
Final Answer: "LangChain, c'est un méfraf pour bâtir des agents puissés par des LLM."

✅ Réponse finale: "LangChain, c'est un méfraf pour bâtir des agents puissés par des LLM."

🎯 Résultat extrait: "LangChain, c'est un méfraf pour bâtir des agents puissés par des LLM."


In [12]:
# Test 4: Recherche avec terme inconnu
result = react_agent("Qu'est-ce que Kubernetes?")
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: Qu'est-ce que Kubernetes?

--- Tour 1/5 ---

Thought: Je vais rechercher des informations sur Kubernetes pour fournir une réponse précise. 
Action: lookup[Kubernetes] 
Observation: [Will be provided by the system after action]

🔍 Recherche: lookup[Kubernetes]
📊 Observation: No info found for Kubernetes

--- Tour 2/5 ---

Thought: Puisque je n'ai pas trouvé d'informations dans la base de connaissances, je vais utiliser mes connaissances pour expliquer ce qu'est Kubernetes. 
Action: skip 
Final Answer: Kubernetes est un système open-source de gestion de conteneurs qui automatise le déploiement, la mise à l'échelle et la gestion d'applications conteneurisées. Il permet de gérer des clusters de machines et de déployer des applications de manière efficace et fiable. Kubernetes est largement utilisé pour orchestrer des conteneurs Docker et offre des fonctionnalités telles que l'équilibrage de charge, la mise à l'échelle automatique et la gestion des configurations.

✅ Réponse fin

## Step 6: question hybride :

Test combinant raisonnement, recherche et calcul.

In [13]:
# Test 5: Question hybride nécessitant plusieurs outils
result = react_agent("If LangChain is a framework, add 10*5 to that info.")
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: If LangChain is a framework, add 10*5 to that info.

--- Tour 1/5 ---

Thought: I need to evaluate the expression 10*5 to provide the correct numerical information along with the fact that LangChain is a framework. 
Action: calculator[10*5] 
Observation: [Will be provided by the system after action]

🔧 Exécution: calculator[10*5]
📊 Observation: 50

--- Tour 2/5 ---

Thought: Now that I have the result of the expression (50), I can combine it with the information about LangChain being a framework. 
Final Answer: LangChain is a framework, and 10 multiplied by 5 equals 50.

✅ Réponse finale: LangChain is a framework, and 10 multiplied by 5 equals 50.

🎯 Résultat extrait: LangChain is a framework, and 10 multiplied by 5 equals 50.


In [14]:
# Test 6: Question hybride plus complexe
#    "Look up what RAG means, then calculate how many letters are in that definition."
result = react_agent(
    "Recherche la signification de RAG, puis calcule le nombre de lettres dans cette définition."
)
print(f"\n🎯 Résultat extrait: {result}")

🤔 Question: Recherche la signification de RAG, puis calcule le nombre de lettres dans cette définition.

--- Tour 1/5 ---

Thought: Je vais d'abord rechercher la signification de "RAG" pour obtenir une définition précise. Ensuite, je pourrai compter le nombre de lettres dans cette définition. 

Action: lookup[RAG]

🔍 Recherche: lookup[RAG]
📊 Observation: Retrieval-Augmented Generation, which uses external knowledge to improve answers.

--- Tour 2/5 ---

Thought: La définition de "RAG" est "Retrieval-Augmented Generation". Je vais maintenant compter le nombre de lettres dans cette définition. 

Action: calculator[30]  # "Retrieval-Augmented Generation" has 30 letters.

🔧 Exécution: calculator[30]
📊 Observation: 30

--- Tour 3/5 ---

Final Answer: La signification de RAG est "Retrieval-Augmented Generation", qui contient 30 lettres.

✅ Réponse finale: La signification de RAG est "Retrieval-Augmented Generation", qui contient 30 lettres.

🎯 Résultat extrait: La signification de RAG est "R

## Step 7: Mini-Projet - Extensions :

Extension de l'agent avec de nouveaux outils (tools).

### 7.1: Ajout de nouveaux outils

In [18]:
from datetime import datetime
import requests

def current_time():
    """
    Retourne l'heure actuelle.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def weather(city: str):
    """
    Stub pour la météo (simulation).
    En production, utiliser une vraie API comme OpenWeatherMap.
    """
    # Simulation de données météo
    weather_data = {
        "Paris": "Partiellement nuageux, 15°C",
        "London": "Pluvieux, 12°C",
        "New York": "Ensoleillé, 20°C",
        "Tokyo": "Dégagé, 18°C"
    }
    return weather_data.get(city, f"Les données météo ne sont pas disponibles actuellement pour {city}")

def search_wikipedia(query: str):
    """
    Recherche simple sur Wikipedia via l'API.
    """
    try:
        url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query.replace(" ", "_")
        response = requests.get(url, timeout=5)

        if response.status_code == 200:
            data = response.json()
            return data.get('extract', 'No summary available')
        else:
            return f"Aucune page Wikipedia trouvée pour : {query}"
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

# Tests des nouveaux outils
print("🧪 Tests des nouveaux outils:")
print(f"current_time(): {current_time()}")
print(f"weather('Paris'): {weather('Paris')}")
print(f"search_wikipedia('Python programming'): {search_wikipedia('Python (programming language)')[:100]}...")

🧪 Tests des nouveaux outils:
current_time(): 2026-02-02 19:45:06
weather('Paris'): Partiellement nuageux, 15°C
search_wikipedia('Python programming'): Aucune page Wikipedia trouvée pour : Python (programming language)...


### 7.2: Agent ReAct étendu

In [19]:
def react_agent_extended(query, max_turns=5, verbose=True):
    """
    Version étendue de l'agent ReAct avec outils supplémentaires.
    """
    system_prompt = """You are an advanced ReAct agent. You must think step by step and take actions when needed.

Format your response as follows:
Thought: [Your reasoning about what to do next]
Action: [tool_name[input]] OR skip if no action needed
Observation: [Will be provided by the system after action]
Final Answer: [Only when you have enough information to answer]

Available tools:
- calculator[expression]: Evaluate mathematical expressions
- lookup[term]: Search for a term in the knowledge base
- current_time[]: Get the current date and time
- weather[city]: Get weather information for a city
- search_wikipedia[query]: Search Wikipedia for information

Rules:
1. Always start with a Thought
2. If you need information, use an Action
3. Wait for Observation before continuing
4. Only provide Final Answer when you're certain
5. You can use multiple actions in sequence if needed
"""

    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]

    if verbose:
        print("="*80)
        print(f"🤔 Question: {query}")
        print("="*80)

    final_answer = None

    for turn in range(max_turns):
        if verbose:
            print(f"\n--- Tour {turn + 1}/{max_turns} ---")

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation,
            temperature=0
        )
        reply = resp.choices[0].message.content

        if verbose:
            print(f"\n{reply}")

        if "Final Answer:" in reply:
            final_answer = reply.split("Final Answer:")[1].strip()
            if verbose:
                print("\n" + "="*80)
                print(f"✅ Réponse finale: {final_answer}")
                print("="*80)
            break

        if "Action:" in reply:
            action_line = [line for line in reply.split("\n") if "Action:" in line][0]
            obs = None

            # Parser les différentes actions
            if "calculator[" in action_line:
                match = re.search(r'calculator\[(.+?)\]', action_line)
                if match:
                    expr = match.group(1)
                    obs = calculator(expr)
                    if verbose:
                        print(f"\n🔧 Exécution: calculator[{expr}]")

            elif "lookup[" in action_line:
                match = re.search(r'lookup\[(.+?)\]', action_line)
                if match:
                    term = match.group(1).strip()
                    obs = lookup(term)
                    if verbose:
                        print(f"\n🔍 Recherche KB: lookup[{term}]")

            elif "current_time" in action_line:
                obs = current_time()
                if verbose:
                    print(f"\n🕒 Heure actuelle")

            elif "weather[" in action_line:
                match = re.search(r'weather\[(.+?)\]', action_line)
                if match:
                    city = match.group(1).strip()
                    obs = weather(city)
                    if verbose:
                        print(f"\n🌤️ Météo: weather[{city}]")

            elif "search_wikipedia[" in action_line:
                match = re.search(r'search_wikipedia\[(.+?)\]', action_line)
                if match:
                    wiki_query = match.group(1).strip()
                    obs = search_wikipedia(wiki_query)
                    if verbose:
                        print(f"\n📚 Wikipedia: search_wikipedia[{wiki_query}]")

            else:
                obs = "Unknown tool. Available: calculator, lookup, current_time, weather, search_wikipedia"

            if obs:
                if verbose:
                    print(f"📊 Observation: {obs}")
                conversation.append({"role": "assistant", "content": reply})
                conversation.append({"role": "user", "content": f"Observation: {obs}"})
        else:
            conversation.append({"role": "assistant", "content": reply})

    if final_answer is None:
        final_answer = "Max turns reached without final answer."

    return final_answer

print("✅ Agent ReAct étendu prêt")

✅ Agent ReAct étendu prêt


### 7.3: Tests de l'agent étendu

In [21]:
# Test 7: Heure actuelle
result = react_agent_extended("Quelle heure est-il maintenant à Paris ?")
print(f"\n🎯 Résultat: {result}")

🤔 Question: Quelle heure est-il maintenant à Paris ?

--- Tour 1/5 ---

Thought: I need to find out the current time in Paris. 
Action: current_time[] 
Observation: [Will be provided by the system after action]

🕒 Heure actuelle
📊 Observation: 2026-02-02 19:47:37

--- Tour 2/5 ---

Thought: The current time is provided as 2026-02-02 19:47:37. Since this is in UTC, I need to convert it to Paris time, which is typically UTC+1, but during daylight saving time (from the last Sunday in March to the last Sunday in October), it is UTC+2. In February, Paris is on standard time (UTC+1). 

Action: skip 
Observation: [No further action needed] 

Final Answer: Il est 20h47 à Paris.

✅ Réponse finale: Il est 20h47 à Paris.

🎯 Résultat: Il est 20h47 à Paris.


In [22]:
# Test 8: Météo
result = react_agent_extended("Quel est la météo à Paris?")
print(f"\n🎯 Résultat: {result}")

🤔 Question: Quel est la météo à Paris?

--- Tour 1/5 ---

Thought: I need to get the current weather information for Paris. 
Action: weather[Paris] 
Observation: [Will be provided by the system after action]

🌤️ Météo: weather[Paris]
📊 Observation: Partiellement nuageux, 15°C

--- Tour 2/5 ---

Thought: I have the weather information for Paris, which is partially cloudy with a temperature of 15°C. 
Final Answer: La météo à Paris est partiellement nuageux avec une température de 15°C.

✅ Réponse finale: La météo à Paris est partiellement nuageux avec une température de 15°C.

🎯 Résultat: La météo à Paris est partiellement nuageux avec une température de 15°C.


In [23]:
# Test 9: Wikipedia
result = react_agent_extended("Recherche des infos dans Wikipedia concernant Apache Kafka")
print(f"\n🎯 Résultat: {result}")

🤔 Question: Recherche des infos dans Wikipedia concernant Apache Kafka

--- Tour 1/5 ---

Thought: I need to search for information about Apache Kafka on Wikipedia to gather relevant details. 
Action: search_wikipedia[Apache Kafka]

📚 Wikipedia: search_wikipedia[Apache Kafka]
📊 Observation: Aucune page Wikipedia trouvée pour : Apache Kafka

--- Tour 2/5 ---

Thought: It seems that there was no Wikipedia page found for "Apache Kafka." This could be due to a search term mismatch or the page might not exist. I will try searching for "Kafka" instead, as it might provide relevant information about the technology. 
Action: search_wikipedia[Kafka]

📚 Wikipedia: search_wikipedia[Kafka]
📊 Observation: Aucune page Wikipedia trouvée pour : Kafka

--- Tour 3/5 ---

Thought: Since there are no Wikipedia pages found for both "Apache Kafka" and "Kafka," I will look up the term "Apache Kafka" in the knowledge base to gather information about it. 
Action: lookup[Apache Kafka]

🔍 Recherche KB: lookup[Ap

In [24]:
# Test 10: Question complexe multi-outils
result = react_agent_extended(
    "Consulte la météo à Tokyo, et si la temperature dépasse 15°C, multiplie-la par 2"
)
print(f"\n🎯 Résultat: {result}")

🤔 Question: Consulte la météo à Tokyo, et si la temperature dépasse 15°C, multiplie-la par 2

--- Tour 1/5 ---

Thought: I need to check the current weather in Tokyo to find out the temperature. After that, I will determine if it exceeds 15°C and perform the multiplication if necessary. 
Action: weather[Tokyo]

🌤️ Météo: weather[Tokyo]
📊 Observation: Dégagé, 18°C

--- Tour 2/5 ---

Thought: The temperature in Tokyo is 18°C, which exceeds 15°C. I will now multiply this temperature by 2. 
Action: calculator[18 * 2]

🔧 Exécution: calculator[18 * 2]
📊 Observation: 36

--- Tour 3/5 ---

Thought: The result of multiplying the temperature by 2 is 36. I have the final answer now. 
Final Answer: 36

✅ Réponse finale: 36

🎯 Résultat: 36


## 📊 Exercices supplémentaires

Pour aller plus loin, essayez ces exercices:

### Exercice 1: Ajouter un outil de conversion

Créez un outil `convert[value, from_unit, to_unit]` pour convertir des unités (km/miles, °C/°F, etc.)

In [ ]:
# Votre code ici
def convert(value: float, from_unit: str, to_unit: str):
    """
    Convertit des valeurs entre différentes unités.
    À compléter...
    """
    pass

# Test
# result = convert(100, "km", "miles")

### Exercice 2: Améliorer le parsing des actions

Créez une fonction `parse_action(reply)` qui retourne un tuple `(tool_name, tool_input)` de manière plus robuste.

In [ ]:
# Votre code ici
def parse_action(reply: str):
    """
    Parse une réponse pour extraire l'outil et son input.
    Returns: (tool_name, tool_input) ou (None, None)
    """
    pass

# Test
# tool, input_val = parse_action("Action: calculator[2+2]")
# print(f"Tool: {tool}, Input: {input_val}")

### Exercice 3: Ajouter des métriques

Modifiez l'agent pour tracker:
- Nombre de tours utilisés
- Outils appelés et leur fréquence
- Temps d'exécution total

In [ ]:
# Votre code ici
import time
from collections import Counter

def react_agent_with_metrics(query, max_turns=5):
    """
    Agent ReAct avec tracking de métriques.
    Returns: (answer, metrics_dict)
    """
    metrics = {
        'turns_used': 0,
        'tools_called': Counter(),
        'execution_time': 0
    }

    start_time = time.time()

    # Votre implémentation ici...

    metrics['execution_time'] = time.time() - start_time

    return "answer", metrics

## 🎓 Récapitulatif du Lab

### Ce que vous avez appris:

1. ✅ **Pattern ReAct**: Alternance entre Thought, Action, Observation
2. ✅ **Boucle agent**: Itération contrôlée avec max_turns
3. ✅ **Parsing d'actions**: Extraction et exécution d'outils
4. ✅ **Gestion de conversation**: Historique contextuel avec OpenAI
5. ✅ **Extension d'outils**: Ajout de nouvelles capacités (Wikipedia, météo, etc.)

### Points clés:

- Le **prompt système** guide le format de raisonnement
- Les **observations** alimentent le contexte pour la prochaine itération
- Le parsing robuste est crucial pour l'exécution fiable
- Les agents peuvent combiner plusieurs outils dans une même session

### Prochaines étapes:

- **Lab 3**: Planning loops et agents complexes
- **Lab 4**: Integration avec LangChain
- **Lab 5**: Multi-agent systems avec CrewAI

---

**🎉 Félicitations! Vous avez implémenté un agent ReAct fonctionnel!**

## 📚 Ressources supplémentaires

- [ReAct Paper (Yao et al., 2022)](https://arxiv.org/abs/2210.03629)
- [OpenAI API Documentation](https://platform.openai.com/docs)
- [LangChain ReAct Agent](https://python.langchain.com/docs/modules/agents/agent_types/react)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)

---

**Version**: 1.0 | **Date**: Février 2026 | **Auteur**: Stéphane Alleaume